In [3]:
from __future__ import print_function
import os.path
import pandas as pd
import gzip
import sys
import numpy as np

sys.path.insert(0, '../..')

from src.CCLE_postp_function import *
from JKBio import Datanalytics as da 
from JKBio import TerraFunction as terra
from JKBio import Helper as h
from gsheets import Sheets
from taigapy import TaigaClient
import dalmatian as dm

from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsClassifier

from bokeh.plotting import *
from bokeh.models import HoverTool
from collections import OrderedDict
from IPython.display import Image,display



%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
tc = TaigaClient()
output_notebook()
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

you need to have installed JKBio in the same folder as ccle_processing


Loading BokehJS ...

In [4]:
samplesetname = "20Q2"
prevname="20Q1"
prevprevname ='19Q4'
prevprevversion= 18
virtual_public='public-20q2-075d'
virtual_dmc='dmc-20q2-2db6'
virtual_internal='internal-20q2-7f46'

workspace1="broad-genomics-delivery/Getz_IBM_CellLines_Exomes"
workspace2="broad-firecloud-ccle/CCLE_DepMap_WES"
workspace3="broad-genomics-delivery/CCLE_DepMap_WES"

workspace6="terra-broad-cancer-prod/CCLE_DepMap_WES"

refworkspace="broad-firecloud-ccle/DepMap_Mutation_Calling_CGA_pipeline"

rnaworkspace="broad-firecloud-ccle/DepMap_hg38_RNAseq"

source1="ibm"
source2="ccle"
source3="ccle"
source6="ccle"
source7="ibm"

refsheet_url = "https://docs.google.com/spreadsheets/d/1XkZypRuOEXzNLxVk9EOHeWRE98Z8_DBvL4PovyM01FE"
sheeturl = "https://docs.google.com/spreadsheets/d/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4"

release = samplesetname

In [ ]:
wm1 = dm.WorkspaceManager(workspace1)
wm2 = dm.WorkspaceManager(workspace2)
wm3 = dm.WorkspaceManager(workspace3)

wm6 = dm.WorkspaceManager(workspace6)

refwm = dm.WorkspaceManager(refworkspace)

In [ ]:
ccle_refsamples = sheets.get(refsheet_url).sheets[0].to_frame()

In [ ]:
! wget https://github.com/mskcc/vcf2maf/archive/v1.6.18.zip

In [6]:
# for rnaseq
#! gsutil cp gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd50046a968/8af15313-7faf-4344-a7b7-97dd52353dc8/RNAseq/0de1da92-8821-4cbd-a9ce-96ed8b06f395/call-MergeVCFs/G41677.MV-4-11.5.g.vcf.gz ../temp/
path= "../temp/G41677.MV-4-11.5.g.vcf.gz"
rnaseq, description = h.vcf_to_df(path)

/home/jeremie/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


{'AD': '"Allelic depths for the ref and alt alleles in the order listed"', 'DP': '"Approximate read depth; some reads may have been filtered"', 'GQ': '"Genotype Quality"', 'GT': '"Genotype"', 'PL': '"Normalized, Phred-scaled likelihoods for genotypes as defined in the VCF specification"', 'AC': '"Allele count in genotypes, for each ALT allele, in the same order as listed"', 'AF': '"Allele Frequency, for each ALT allele, in the same order as listed"', 'AN': '"Total number of alleles in called genotypes"', 'BaseQRankSum': '"Z-score from Wilcoxon rank sum test of Alt Vs. Ref base qualities"', 'DB': '"dbSNP Membership"', 'ExcessHet': '"Phred-scaled p-value for exact test of excess heterozygosity"', 'FS': '"Phred-scaled p-value using Fisher\'s exact test to detect strand bias"', 'InbreedingCoeff': '"Inbreeding coefficient as estimated from the genotype likelihoods per-sample when compared against the Hardy-Weinberg expectation"', 'MLEAC': '"Maximum likelihood expectation (MLE) for the allel

In [7]:
# for rnaseq filtered
#!gsutil cp gs://fc-secure-639c94ba-2b0d-4960-92fc-9cd50046a968/8af15313-7faf-4344-a7b7-97dd52353dc8/RNAseq/0de1da92-8821-4cbd-a9ce-96ed8b06f395/call-VariantFiltration/G41677.MV-4-11.5.variant_filtered.vcf.gz ../temp/
rnaseqfilt, description = h.vcf_to_df("../temp/G41677.MV-4-11.5.variant_filtered.vcf.gz")

{'AD': '"Allelic depths for the ref and alt alleles in the order listed"', 'DP': '"Approximate read depth; some reads may have been filtered"', 'GQ': '"Genotype Quality"', 'GT': '"Genotype"', 'PL': '"Normalized, Phred-scaled likelihoods for genotypes as defined in the VCF specification"', 'AC': '"Allele count in genotypes, for each ALT allele, in the same order as listed"', 'AF': '"Allele Frequency, for each ALT allele, in the same order as listed"', 'AN': '"Total number of alleles in called genotypes"', 'BaseQRankSum': '"Z-score from Wilcoxon rank sum test of Alt Vs. Ref base qualities"', 'DB': '"dbSNP Membership"', 'ExcessHet': '"Phred-scaled p-value for exact test of excess heterozygosity"', 'FS': '"Phred-scaled p-value using Fisher\'s exact test to detect strand bias"', 'InbreedingCoeff': '"Inbreeding coefficient as estimated from the genotype likelihoods per-sample when compared against the Hardy-Weinberg expectation"', 'MLEAC': '"Maximum likelihood expectation (MLE) for the allel

In [8]:
# for unfiltered wes
#!gsutil cp gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/51fcb10c-9a67-4ff9-b13c-d9667e2d49d0/HaplotypeCallerGvcf_GATK4/6611b28a-12c2-484e-a4a3-901708785f95/call-MergeGVCFs/SANGER_MV411_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE.g.vcf.gz ../temp/
wes, description = h.vcf_to_df("../temp/SANGER_MV411_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE.g.vcf.gz")

{'AD': '"Allelic depths for the ref and alt alleles in the order listed"', 'DP': '"Approximate read depth; some reads may have been filtered"', 'GQ': '"Genotype Quality"', 'GT': '"Genotype"', 'MIN_DP': '"Minimum DP observed within the GVCF block"', 'PGT': '"Physical phasing haplotype information, describing how the alternate alleles are phased in relation to one another"', 'PID': '"Physical phasing ID information, where each unique ID within a given sample (but not across samples) connects records within a phasing group"', 'PL': '"Normalized, Phred-scaled likelihoods for genotypes as defined in the VCF specification"', 'SB': '"Per-sample component statistics which comprise the Fisher\'s Exact Test to detect strand bias."', 'BaseQRankSum': '"Z-score from Wilcoxon rank sum test of Alt Vs. Ref base qualities"', 'ClippingRankSum': '"Z-score From Wilcoxon rank sum test of Alt vs. Ref number of hard clipped bases"', 'DS': '"Were any of the samples downsampled?"', 'END': '"Stop position of th

In [9]:
# for filtered wes
#! gsutil cp gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/d953cc3f-7941-4426-bb44-a41992d57606/Cram2FilteredVcf/dc943e9d-aa73-4314-b43d-208a80319997/call-FilterVariantTranches/CDS-FInR9b_cnn_filtered.vcf.gz ../temp/
wesfilt, description = h.vcf_to_df("../temp/CDS-FInR9b_cnn_filtered.vcf.gz")

{'AD': '"Allelic depths for the ref and alt alleles in the order listed"', 'DP': '"Approximate read depth; some reads may have been filtered"', 'GQ': '"Genotype Quality"', 'GT': '"Genotype"', 'PL': '"Normalized, Phred-scaled likelihoods for genotypes as defined in the VCF specification"', 'AC': '"Allele count in genotypes, for each ALT allele, in the same order as listed"', 'AF': '"Allele Frequency, for each ALT allele, in the same order as listed"', 'AN': '"Total number of alleles in called genotypes"', 'BaseQRankSum': '"Z-score from Wilcoxon rank sum test of Alt Vs. Ref base qualities"', 'CNN_1D': '"Log odds of being a true variant versus being false under the trained 1D Convolutional Neural Network"', 'DS': '"Were any of the samples downsampled?"', 'ExcessHet': '"Phred-scaled p-value for exact test of excess heterozygosity"', 'FS': '"Phred-scaled p-value using Fisher\'s exact test to detect strand bias"', 'InbreedingCoeff': '"Inbreeding coefficient as estimated from the genotype lik

In [10]:
# for somatic filtered wes
somwesfilt = pd.read_csv('../temp/mutations.'+release+'.all.tsv', sep='\t')
somwesfilt = somwesfilt[somwesfilt.DepMap_ID=="ACH-000045"]
somwesfilt = somwesfilt.rename(columns=h.rename_mut)

/home/jeremie/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (19,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
# for somatic unfiltered wes
#!gsutil cp gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/9e3cc501-3f08-47fb-87a5-0359febb833c/CGA_Production_Analysis_Workflow/72efd16d-fa2d-4d78-b7c7-81ec2a6d08e9/call-Oncotate_Task/CDS-FInR9b.MuTect1_SNV.MuTect2_INDEL.Strelka_INDEL.annotated.maf ../temp/
somwes = pd.read_csv("../temp/CDS-FInR9b.MuTect1_SNV.MuTect2_INDEL.Strelka_INDEL.annotated.maf", sep='\t')
somwes = somwes.rename(columns=h.rename_mut)

In [56]:
somwes.columns.tolist()

['Hugo_Symbol',
 'Entrez_Gene_Id',
 'Center',
 'NCBI_Build',
 'chr',
 'pos',
 'End_position',
 'Strand',
 'Variant_Classification',
 'Variant_Type',
 'ref',
 'alt',
 'Tumor_Seq_Allele2',
 'dbSNP_RS',
 'dbSNP_Val_Status',
 'Tumor_Sample_Barcode',
 'Matched_Norm_Sample_Barcode',
 'Match_Norm_Seq_Allele1',
 'Match_Norm_Seq_Allele2',
 'Tumor_Validation_Allele1',
 'Tumor_Validation_Allele2',
 'Match_Norm_Validation_Allele1',
 'Match_Norm_Validation_Allele2',
 'Verification_Status',
 'Validation_Status',
 'Mutation_Status',
 'Sequencing_Phase',
 'Sequence_Source',
 'Validation_Method',
 'Score',
 'BAM_file',
 'Sequencer',
 'dbNSFP_aapos_SIFT',
 'n_ref_count',
 'HGNC_Ensembl Gene ID',
 'dbNSFP_phyloP46way_placental_rankscore',
 'HGVS_protein_change',
 'dbNSFP_MutationTaster_score',
 'TAR',
 'ESP_AA_GTC',
 'dbNSFP_Ancestral_allele',
 'ESP_EA_AGE',
 '1000gp3_SVTYPE',
 'gencode_transcript_name',
 'UniProt_Site',
 'ClinVar_SYM',
 'ESP_Chromosome',
 '1000gp3_SVLEN',
 'dbNSFP_phastCons46way_primate

In [12]:
# for strelka wes
#!gsutil cp gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/9e3cc501-3f08-47fb-87a5-0359febb833c/CGA_Production_Analysis_Workflow/72efd16d-fa2d-4d78-b7c7-81ec2a6d08e9/call-Strelka/CDS-FInR9b.all.somatic.snvs.vcf ../temp/
strelka, description = h.vcf_to_df("../temp/CDS-FInR9b.all.somatic.snvs.vcf", hasfilter=True,samples=['sample1_A','sample1_B'])
#!gsutil cp gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/9e3cc501-3f08-47fb-87a5-0359febb833c/CGA_Production_Analysis_Workflow/72efd16d-fa2d-4d78-b7c7-81ec2a6d08e9/call-Strelka/CDS-FInR9b.all.somatic.indels.vcf ../temp/
strelka2, description = h.vcf_to_df("../temp/CDS-FInR9b.all.somatic.indels.vcf",samples=['sample1_A','sample1_B'],hasfilter=True)
strelka = pd.concat([strelka,strelka2], ignore_index=True,sort=False).sort_values(by=['chr','pos'])

{'QSS': '"Quality score for any somatic snv, ie. for the ALT allele to be present at a significantly different frequency in the tumor and normal"', 'TQSS': '"Data tier used to compute QSS"', 'NT': '"Genotype of the normal in all data tiers, as used to classify somatic variants. One of {ref,het,hom,conflict}."', 'QSS_NT': '"Quality score reflecting the joint probability of a somatic variant and NT"', 'TQSS_NT': '"Data tier used to compute QSS_NT"', 'SGT': '"Most likely somatic genotype excluding normal noise states"', 'SOMATIC': '"Somatic mutation"', 'DP': '"Read depth for tier1 (used+filtered)"', 'FDP': '"Number of basecalls filtered from original read depth for tier1"', 'SDP': '"Number of reads with deletions spanning this site at tier1"', 'SUBDP': '"Number of reads below tier1 mapping quality threshold aligned across this site"', 'AU': '"Number of \'A\' alleles used in tiers 1,2"', 'CU': '"Number of \'C\' alleles used in tiers 1,2"', 'GU': '"Number of \'G\' alleles used in tiers 1,2"

In [14]:
# for mutect1 wes
#!gsutil cp gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/9e3cc501-3f08-47fb-87a5-0359febb833c/CGA_Production_Analysis_Workflow/72efd16d-fa2d-4d78-b7c7-81ec2a6d08e9/call-Gather_Task/CDS-FInR9b.MuTect1.call_stats.txt ../temp/
mutect1 = pd.read_csv('../temp/CDS-FInR9b.MuTect1.call_stats.txt', sep='\t', header=1, index_col=None)
mutect1 = mutect1.rename(columns=h.rename_mut)

/home/jeremie/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
# for mutect 2 wes
#!gsutil cp gs://fc-secure-012d088c-f039-4d36-bde5-ee9b1b76b912/9e3cc501-3f08-47fb-87a5-0359febb833c/CGA_Production_Analysis_Workflow/72efd16d-fa2d-4d78-b7c7-81ec2a6d08e9/call-Gather_Task/CDS-FInR9b.MuTect2.call_stats.vcf ../temp/
mutect2, description = h.vcf_to_df('../temp/CDS-FInR9b.MuTect2.call_stats.vcf', hasfilter=True)

{'AD': '"Allelic depths for the ref and alt alleles in the order listed"', 'AF': '"Allele fractions of alternate alleles in the tumor"', 'DP': '"Approximate read depth; some reads may have been filtered"', 'F1R2': '"Count of reads in F1R2 pair orientation supporting each allele"', 'F2R1': '"Count of reads in F2R1 pair orientation supporting each allele"', 'GQ': '"Genotype Quality"', 'GT': '"Genotype"', 'MBQ': '"median base quality"', 'MFRL': '"median fragment length"', 'MMQ': '"median mapping quality"', 'MPOS': '"median distance from end of read"', 'PGT': '"Physical phasing haplotype information, describing how the alternate alleles are phased in relation to one another"', 'PID': '"Physical phasing ID information, where each unique ID within a given sample (but not across samples) connects records within a phasing group"', 'PL': '"Normalized, Phred-scaled likelihoods for genotypes as defined in the VCF specification"', 'SA_MAP_AF': '"MAP estimates of allele fraction given z"', 'SA_POST

In [15]:
# for unfiltered wgs
# for filtered wgs
#! gsutil cp gs://fc-secure-bd7b8bc9-f665-4269-997e-5a402088a369/77dbe69a-509c-4618-a2bf-f809c1aea47a/liftover_workflows/2950a03d-c092-4355-b261-2e6eb50fca41/call-liftover/ccle_ACH-000045_liftover.vcf ../temp/
wgsfilt, description = h.vcf_to_df('../temp/ccle_ACH-000045_liftover.vcf')
wgsfilt.chr = [i[3:] for i in wgsfilt.chr]

{'AD': '"Allelic depths for the ref and alt alleles in the order listed"', 'DP': '"Approximate read depth; some reads may have been filtered"', 'GQ': '"Genotype Quality"', 'GT': '"Genotype"', 'PL': '"Normalized, Phred-scaled likelihoods for genotypes as defined in the VCF specification"', 'AC': '"Allele count in genotypes, for each ALT allele, in the same order as listed"', 'AF': '"Allele Frequency, for each ALT allele, in the same order as listed"', 'AN': '"Total number of alleles in called genotypes"', 'BaseQRankSum': '"Z-score from Wilcoxon rank sum test of Alt Vs. Ref base qualities"', 'CNN_1D': '"Log odds of being a true variant versus being false under the trained 1D Convolutional Neural Network"', 'DS': '"Were any of the samples downsampled?"', 'END': '"Stop position of the interval"', 'ExcessHet': '"Phred-scaled p-value for exact test of excess heterozygosity"', 'FS': '"Phred-scaled p-value using Fisher\'s exact test to detect strand bias"', 'InbreedingCoeff': '"Inbreeding coef

In [29]:
data= {'wgsfilt': wgsfilt, 'mutect2': mutect2, 'mutect1': mutect1, 'strelka': strelka, 'somwes': somwes, 'somwesfilt': somwesfilt, 'wes': wes, 'rnaseq': rnaseq, 'rnaseqfilt': rnaseqfilt, 'wesfilt': wesfilt}

In [41]:
mutset = {}
for i,v in data.items():
    chrom = v.chr.tolist()
    alt = v.alt.tolist()
    pos = v.pos.tolist()
    mutset[i] = set([str(chrom[i])+'_'+str(pos[i]) for i, a in enumerate(alt)])

In [ ]:
len(mutset['wgsfilt'] & mutset['wesfilt']) / len(mutset['wesfilt']), len(mutset['wgsfilt']), len(mutset['wesfilt'])

In [ ]:
len(mutset['rnaseqfilt']  & mutset['wesfilt']) / len(mutset['wesfilt'])

In [ ]:
len(mutset['rnaseq']  & mutset['wesfilt']) / len(mutset['wesfilt'])

In [ ]:
len(mutset['rnaseqfilt']  & mutset['rnaseq']) / len(mutset['rnaseq'])

In [ ]:
len(mutset['wes']  & mutset['wesfilt']) / len(mutset['wesfilt'])

In [ ]:
len(mutset['wes']  & mutset['wesfilt']) / len(mutset['wes'])
# how many mutations is the cnn filtering..

In [ ]:
len(mutset['wesfilt']), len(mutset['wes'])

In [ ]:
len(mutset['strelka']), len(mutset['mutect1']), len(mutset['mutect2'])
# how many mutations each contain and their similarity between each caller in cga

In [ ]:
len(mutset['strelka'] & mutset['mutect1'] & mutset['mutect2'])

In [ ]:
len(mutset['somwes']), len(mutset['somwesfilt']), len(mutset['somwesfilt'] & mutset['somwes'])
# how many mutations each contain and their similarity between cga middle and cga end

In [ ]:
len(mutset['somwes'] & mutset['mutect1']) / len(mutset['mutect1']), len(mutset['somwesfilt'] & mutset['mutect1']) / len(mutset['mutect1'])

In [ ]:
len(mutset['somwes'] & mutset['mutect2']) / len(mutset['mutect2']), len(mutset['somwesfilt'] & mutset['mutect2']) / len(mutset['mutect2'])

In [ ]:
len(mutset['somwes'] & mutset['strelka']) / len(mutset['strelka']), len(mutset['somwesfilt'] & mutset['strelka']) / len(mutset['strelka'])

In [ ]:
len(mutset['somwes'] & mutset['wes']) / len(mutset['somwes']), len(mutset['wes'])

In [ ]:
len(mutset['somwes'] & mutset['wesfilt']) / len(mutset['somwes']), len(mutset['wesfilt'])

In [ ]:
len(mutset['somwesfilt'] & mutset['wesfilt']) / len(mutset['somwesfilt']), len(mutset['wesfilt'])
# good agreement between wes hapcall and wes cga and we can see how much we recover once bad mutations get filtered!

In [ ]:
len(mutset['somwes'] & mutset['wesfilt']), len(mutset['somwesfilt'] & mutset['wesfilt'])

In [ ]:
len(mutset['somwes'] & mutset['wgsfilt']) / len(mutset['somwes'])

In [42]:
len(mutset['somwesfilt'] & mutset['wgsfilt']) / len(mutset['somwesfilt'])
# good agreement between wgs hapcall and wes cga

0.8333333333333334

17% of the mutations in WES could not be found in WGS. this is explained by read depth issues?
--> artifacts? low coverage wgs.

In [ ]:
loc= [176551, 1170265, 825909, 1098957, 826976, 278207, 391815] # 7 that we would have liked to find and likely 3 False positives from the previous somatic mutation pipeline. The rest could not have been caught on this experiment

In [51]:
res = subset(data['somwesfilt'],mutset['somwesfilt'] - mutset['wgsfilt'])
res

,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,chr,pos,End_position,Strand,Variant_Classification,Variant_Type,ref,...,ExAC_AF,CGA_WES_AC,SangerWES_AC,SangerRecalibWES_AC,RNAseq_AC,HC_AC,RD_AC,WGS_AC,Variant_annotation,DepMap_ID
338957,EIF5B,9669,37,2,100015269,100015270,+,Frame_Shift_Ins,INS,-,...,NaN,NaN,NaN,NaN,69:314,NaN,NaN,NaN,damaging,ACH-000045
678687,MT-ATP6,4508,37,M,9150,9150,+,Silent,SNP,A,...,NaN,NaN,NaN,45:1,NaN,NaN,NaN,NaN,silent,ACH-000045
1018133,SLC22A16,85413,37,6,110763879,110763880,+,Frame_Shift_Ins,INS,-,...,0.000016,NaN,NaN,NaN,11:60,NaN,NaN,NaN,damaging,ACH-000045
444857,GOLGA8I,283796,37,15,23261942,23261942,+,Missense_Mutation,SNP,A,...,0.000034,NaN,16:73,16:90,NaN,NaN,NaN,NaN,other non-conserving,ACH-000045
622626,LTN1,26046,37,21,30303518,30303519,+,Frame_Shift_Ins,INS,-,...,NaN,NaN,NaN,NaN,16:76,NaN,NaN,NaN,damaging,ACH-000045
296745,DLGAP2,9228,37,8,1628713,1628713,+,Splice_Site,SNP,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12:35,damaging,ACH-000045
445004,GOLGA8R,101059918,37,15,30702129,30702129,+,Missense_Mutation,SNP,A,...,NaN,NaN,NaN,NaN,16:1,NaN,NaN,NaN,other non-conserving,ACH-000045
883766,PRDX6,9588,37,1,173456921,173456922,+,Frame_Shift_Ins,INS,-,...,NaN,NaN,NaN,NaN,37:264,NaN,NaN,NaN,damaging,ACH-000045
930038,RBM42,79171,37,19,36120079,36120079,+,Silent,SNP,G,...,NaN,NaN,NaN,4:8,NaN,NaN,NaN,NaN,silent,ACH-000045
871718,POU6F2,11281,37,7,39379285,39379285,+,Missense_Mutation,SNP,C,...,0.000042,NaN,NaN,47:55,NaN,NaN,NaN,NaN,other non-conserving,ACH-000045


In [ ]:
locify(mutset['somwesfilt'] - mutset['wgsfilt'])

In [ ]:
len(mutset['somwesfilt'] & mutset['wes']) / len(mutset['somwesfilt']), len(mutset['wes'])
# good agreement between wes hapcall and wes cga

In [ ]:
#we don't have a good similarity wes/wgs, can it be because of slight differences in the base distances? can it be because of the mutation caller?
len(set([i for i in mutset['wesfilt']]) & set([i for i in mutset['wgsfilt']]))/len(mutset['wesfilt'])

In [49]:
def locify(dat):
    return ['chr'+d.split('_')[0]+':'+str(int(d.split('_')[1])-40)+'-'+str(int(d.split('_')[1])+40) for d in dat]

In [ ]:
'chr6:27420132-27420212':'rna',
'chr5:32089849-32089929':'del',
'chr4:100574667-100574747':'wgs',
'chr7:100643254-100643334':'wgs',
'chr15:89070794-89070874':'wgs',
'chr19:36120039-36120119':'snp',
'chr20:31196288-31196368':'snp',
'chr15:42343066-42343146':False,
'chr11:6568654-6568734':'snp',

'chr9:24543236-24543316':'snp',
'chr15:23261902-23261982':'snp',
'chr4:84255808-84255888':True,
'chr13:76055619-76055699':'snp',
'chrX:18969242-18969322':,
'chr4:36152613-36152693':,
'chr19:50354695-50354775':,
'chr4:100574668-100574748':,
'chr6:110763839-110763919':,
'chr7:100648111-100648191':,
'chr5:70858221-70858301':,
'chr3:120489593-120489673':,
'chr2:27601639-27601719':,
'chr4:77184865-77184945':,

In [44]:
locify(mutset['somwesfilt'] - mutset['wesfilt'])

NameError: name 'locify' is not defined

In [24]:
def subset(df,mut):
    return pd.concat([df[(df.chr==val.split('_')[0]) & (df.pos==int(val.split('_')[1]))] for val in mut])

In [52]:
a = subset(data['somwesfilt'],mutset['somwesfilt'] - mutset['wesfilt'])
print(60-a.SangerRecalibWES_AC.isna().sum())
a[~a.SangerRecalibWES_AC.isna()]

20


,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,chr,pos,End_position,Strand,Variant_Classification,Variant_Type,ref,...,ExAC_AF,CGA_WES_AC,SangerWES_AC,SangerRecalibWES_AC,RNAseq_AC,HC_AC,RD_AC,WGS_AC,Variant_annotation,DepMap_ID
678687,MT-ATP6,4508,37,M,9150,9150,+,Silent,SNP,A,...,NaN,NaN,NaN,45:1,NaN,NaN,NaN,NaN,silent,ACH-000045
444857,GOLGA8I,283796,37,15,23261942,23261942,+,Missense_Mutation,SNP,A,...,0.000034,NaN,16:73,16:90,NaN,NaN,NaN,NaN,other non-conserving,ACH-000045
930038,RBM42,79171,37,19,36120079,36120079,+,Silent,SNP,G,...,NaN,NaN,NaN,4:8,NaN,NaN,NaN,NaN,silent,ACH-000045
176551,CCDC65,85478,37,12,49312156,49312157,+,Frame_Shift_Del,DEL,CA,...,0.000016,NaN,40:8,49:74,12:10,NaN,NaN,11:17,damaging,ACH-000045
309536,DNHD1,144132,37,11,6568694,6568694,+,Silent,SNP,C,...,NaN,NaN,29:29,30:29,22:24,NaN,NaN,26:17,silent,ACH-000045
1170265,TROVE2,6738,37,1,193038192,193038192,+,Frame_Shift_Del,DEL,A,...,NaN,NaN,12:7,21:30,NaN,NaN,NaN,6:20,damaging,ACH-000045
78368,ARHGEF16,27237,37,1,3380076,3380076,+,Missense_Mutation,SNP,G,...,NaN,NaN,5:4,5:4,NaN,NaN,NaN,26:18,other non-conserving,ACH-000045
956348,RP11-410N8.4,0,37,20,31196328,31196328,+,Missense_Mutation,SNP,A,...,NaN,NaN,4:6,4:7,NaN,NaN,NaN,16:23,other non-conserving,ACH-000045
343760,ENAH,55740,37,1,225707079,225707079,+,Missense_Mutation,SNP,T,...,NaN,NaN,NaN,6:4,NaN,NaN,NaN,NaN,other non-conserving,ACH-000045
1098957,TAF3,83860,37,10,8005934,8005934,+,Frame_Shift_Del,DEL,T,...,NaN,NaN,25:7,32:43,7:16,NaN,NaN,5:25,damaging,ACH-000045


####  we have 60-40=20 that the old pipeline has found but not us. of these 20; 7 were suspicious as they were not confirmed by any other sequencing methods, had low depth etc.

In [53]:
res = subset(data['somwesfilt'],mutset['somwesfilt'] & mutset['wesfilt'])
print(res.SangerRecalibWES_AC.isna().sum())
res[res.SangerRecalibWES_AC.isna()]

16


,Hugo_Symbol,Entrez_Gene_Id,NCBI_Build,chr,pos,End_position,Strand,Variant_Classification,Variant_Type,ref,...,ExAC_AF,CGA_WES_AC,SangerWES_AC,SangerRecalibWES_AC,RNAseq_AC,HC_AC,RD_AC,WGS_AC,Variant_annotation,DepMap_ID
527683,ISOC1,51015,37,5,128430479,128430479,+,Missense_Mutation,SNP,C,...,NaN,NaN,NaN,NaN,47:47,NaN,NaN,11:7,other non-conserving,ACH-000045
729754,NEURL,0,37,10,105344322,105344322,+,Missense_Mutation,SNP,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7:10,other non-conserving,ACH-000045
335550,EHMT1,79813,37,9,140707977,140707977,+,Missense_Mutation,SNP,C,...,NaN,NaN,12:12,NaN,147:142,NaN,NaN,18:17,other non-conserving,ACH-000045
138731,C1orf122,127687,37,1,38274362,38274362,+,Silent,SNP,A,...,0.000010,NaN,NaN,NaN,102:85,NaN,NaN,14:10,silent,ACH-000045
247398,CPVL,54504,37,7,29035415,29035415,+,Silent,SNP,T,...,NaN,NaN,20:29,NaN,NaN,NaN,NaN,17:12,silent,ACH-000045
385824,FBLN2,2199,37,3,13655514,13655514,+,Missense_Mutation,SNP,C,...,NaN,NaN,14:22,NaN,NaN,NaN,NaN,11:16,other non-conserving,ACH-000045
1172171,TRPM5,29850,37,11,2434364,2434364,+,Splice_Site,SNP,C,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20:17,damaging,ACH-000045
1242740,XRCC3,7517,37,14,104169511,104169511,+,Splice_Site,SNP,A,...,NaN,NaN,NaN,NaN,112:106,NaN,NaN,9:16,damaging,ACH-000045
335548,EHMT1,79813,37,9,140707976,140707976,+,Silent,SNP,T,...,0.000016,NaN,11:12,NaN,149:139,NaN,NaN,18:17,silent,ACH-000045
291218,DHX36,170506,37,3,154018843,154018843,+,Nonsense_Mutation,SNP,C,...,NaN,NaN,46:45,NaN,NaN,NaN,NaN,8:24,damaging,ACH-000045


In [ ]:
# we have found 16 mutations that the old pipeline did not fouund, of these, only one was only confirmed by one sequencing method

In [ ]:
subset(data['somwesfilt'],mutset['rnaseqfilt'] & mutset['somwesfilt']).RNAseq_AC.isna().sum() #113

In [ ]:
subset(data['somwesfilt'],mutset['somwesfilt'] - mutset['rnaseqfilt']).RNAseq_AC.isna().sum() #113

In [ ]:
# we have found 12 more rnaseq mutations and lost 9 rnaseq mutations

In [47]:
res = subset(data['somwesfilt'],mutset['somwesfilt'] - mutset['wgsfilt'])
print(37-res.WGS_AC.isna().sum())

AttributeError: 'DataFrame' object has no attribute 'WGS_AC'

In [ ]:
res

### we have high allele freq mut which seems it might be that they get filtered by the cnn. two seemed quite shady